In [1]:
import numpy as np
import os
import time
import PIL
import PIL.Image
import tensorflow as tf
from keras import layers
from tensorflow import keras
import pathlib

img_height = 180
img_width = 180

checkpoint_path = "training/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "./training/training-data/whiteboard",
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=32,
  image_size=(img_height, img_width))

class_names = train_ds.class_names

Found 890 files belonging to 2 classes.
Using 712 files for training.


In [3]:
print(class_names)

['etc', 'whiteboard']


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  "./training/training-data/whiteboard",
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=32,
  image_size=(img_height, img_width))

Found 890 files belonging to 2 classes.
Using 178 files for validation.


In [5]:

normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))



num_classes = len(class_names)

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

traino_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)


C:\Users\Vile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model = keras.Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu', input_shape=[5]),
  layers.Dense(num_classes),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

C:\Users\Vile\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15,
)

Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.8471 - loss: 0.5226 - val_accuracy: 0.9270 - val_loss: 0.1263
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 291ms/step - accuracy: 0.9606 - loss: 0.1642 - val_accuracy: 0.9494 - val_loss: 0.0994
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 283ms/step - accuracy: 0.9652 - loss: 0.1004 - val_accuracy: 0.8933 - val_loss: 0.2426
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 284ms/step - accuracy: 0.9581 - loss: 0.1162 - val_accuracy: 0.9607 - val_loss: 0.0935
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 7s 287ms/step - accuracy: 0.9784 - loss: 0.0660 - val_accuracy: 0.9607 - val_loss: 0.1283
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 407ms/step - accuracy: 0.9730 - loss: 0.0936 - val_accuracy: 0.9326 - val_loss: 0.2239
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 425ms/step - accuracy: 0.9637 - loss: 0.0725 - val_accuracy: 0.9551 - val_loss: 0.1009
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 440ms/step - accuracy: 0.9869 - loss: 0.0455 - val_accuracy: 0.

In [11]:
result = time.localtime()
date = ("{}-{}_{}-{}".format(result.tm_hour,result.tm_min,result.tm_mday,result.tm_mon))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.save('./training/my_model_{}.keras'.format(date))
